# 实验二：用户及文件权限管理
___


## 实验目的与内容

### 实验目的

1. 掌握超级用户创建、删除用户、用户组等操作
2. 文件权限的设置和管理

### 实验内容

- [1. Linux用户管理](#1.-Linux用户管理)

    * [1.1 查看用户](#1.1-查看用户)
    * [1.2 创建用户](#1.2-创建用户)
    * [1.3 关于用户组](#1.3-关于用户组)
    * [1.4 删除用户](#1.4-删除用户)
- [上机练习1](#$\S$-上机练习1)

- [2. 文件权限管理](#2.-文件权限管理)

    * [2.1 查看文件权限](#2.1-查看文件权限)
    * [2.2 修改文件权限](#2.2-修改文件权限)
    * [2.3 修改文件属主](#2.3-修改文件属主)

- [上机练习2](#$\S$-上机练习2)

## 1. Linux用户管理

Linux是实现多用户登录的系统。多个用户同时登录一台主机，共享主机资源的同时，也具有各自独立的用户空间。Linux的**用户管理**和**权限机制**能实现用户空间之间的相互隔离。

### 1.1 查看用户

每个Linux用户具有唯一的用户ID、用户名以及对应的用户目录、用户默认的登录方式（shell）。用户的基本设置在文件`/etc/passwd`中定义：
```
root:x:0:0:root:/root:/bin/bash
bin:x:1:1:bin:/bin:/sbin/nologin
bio:x:1000:1000:bio:/home/bio:/bin/bash
```
该文件用`:`分隔成7列：
- 第1列：用户名 （username）
- 第2列：用户密码，用`x`表示 （encrypted password）
- 第3列：用户ID （user ID）
- 第4列：用户所属的组ID （group ID）
- 第5列：用户描述（user description）
- 第6列：用户家目录（home directory）
- 第7列：用户登录的默认shell（default login shell）

#### `whoami`

`whoami`命令返回当前登录用户的ID，登录的伪终端（Pseudo terminal, pts）以及登录的时间等信息：
```
bio      pts/0        2016-12-22 12:49 (:0)
```
- 登录用户的用户名为`bio`；
- 登录的伪终端为`pts/0`；所谓伪终端，是相对于`/dev/tty[1-7]`的`终端`来说的，是由`/dev/tty7`发起的
- 登录时间为`2016-12-22 12:49`，从`:0`(本机X-window)登录。

#### `who`命令常用参数

| 参数 | 说明 | 举例 |
| --- | --- | --- |
| `-a` | 输出所有信息 | `who -a` |
| `-d` | 输出死掉的进程 | `who -d` |
| `-m` | 同`who am i` | `who -m` |
| `-q` | 输出当前登录用户 | `who -q` |
| `-u` | 输出当前登录用户的所有登录信息 | `who -u` |
| `-r` | 输出当前运行等级（running level） | `who -r` |


### 1.2 创建用户

在 Linux 系统里， `root`用户是 Linux 和 UNIX 系统中的超级管理员帐户，拥有整个系统至高无上的权力，可操作所有对象。所以很多黑客在入侵系统的时候，都要把权限提升到 root 权限。

一般在登录系统时都是以普通用户的账户登录的，创建其他账户需要`root`权限，需要用到`sudo`命令。但这必须有两个大前提：
- 知道当前登录用户的密码；
- 当前用户必须在`sudoer`用户组，这是定义在`/etc/sudoers`配置文件中


#### `su`、`su -` 和 `sudo`命令

- `su <user>`切换到目标用户`<user>`，需要用到`<user>`的密码；
- `su - <user>`切换到目标用户`<user>`，并加载该目标用户的环境变量；
- `sudo <command>`采用特权用户级别执行`<command>`命令，但需要当前用户属于`sudo`用户组，且需输入当前用户的密码。

- 切换用户后如果要退出，输入`exit`或者按下快捷键`CTRL+d`。

#### `sudo useradd <user>`

- **添加组**：`sudo groupadd <group>`，其中`<group>`为组名
- **添加用户**：`sudo useradd <user> -g <group> -d <home>`，其中`<group>`为组名，`<home>`为用户工作目录
- **设置密码**：`sudo passwd <user>`

### 1.3 关于用户组

用户组简单来说，就是一组用户的集合，共享一些资源和权限，同时也具有私有资源。用户组定义在文件`/etc/group`中。

- Linux下每个用户都至少属于一个用户组
- Linux用户可属于多个不同的用户组

#### 查看用户所属组：`groups`

`groups bio`命令返回用户`bio`所属的所有组的名称：
```
bio : bio wheel
```

其中`bio`是默认的组，而`wheel`是一个`sudoers`的组，定义在/etc/sudoers文件中：
```
## Allows people in group wheel to run all commands
%wheel	ALL=(ALL)	ALL
```

如果需要该用户组不需要输入密码就能运行`sudo <cmd>`，则只需要修改该文件：
```
## Allow users in group wheel to run all commands without entering password
%wheel                ALL=(ALL)                                   NOPASSWORD: ALL
```

#### 运行`usermod`修改用户所在用户组、修改家目录

```bash
usermod -G wheel bio
```
为用户添加所属组。

### 1.4 删除用户

用`userdel <user>`命令可快速删除系统中的用户`user`。


### <font color="red">$\S$ 上机练习1</font>

1. 为本机添加用户组`scientist` 
2. 在目录`/etc/sudoers.d`目录下添加文件`scientist`，在其中添加内容：
```
%scientist          ALL=(ALL)           ALL
```
其意义是什么？
3. 为本机添加用户`bio2`，使用默认的家目录和默认的登录shell以及对应的用户组
4. 该用户能否运行`sudo <cmd>`？如果不能，为什么？
5. 修改用户`bio2`的所属家目录为`/share/home/bio`，并为该用户添加附加用户组`scientist`
6. 如今该用户是否能运行`sudo <cmd>`？为什么？
7. 列出用户家目录下的所有文件，分别是什么？
8. 删除该用户`bio2`及其家目录的所有文件。

## 2. 文件权限管理

在Linux下所有的内容皆文件，因此文件的权限就显得尤为重要了。

### 2.1 查看文件权限

我们可用`ls -l`输出目录内容的长格式，即可查看文件的权限：
```
drwxr-xr-x.  5 bio bio             60    12月 18  06:54 Desktop
drwxr-xr-x.  2 bio bio        4096       1月   9  09:44 Downloads
- rwxr-xr-x.  1 bio bio 6062231    11月 15 14:45 plink
```

这里从左至右依次列出的分别是：
- 第一列：文件权限（file privileges），其中第1位表示的是文件类型（d-目录；- - 普通文件；l - 符号链接文件；b - 块设备文件；c - 字符设备文件；s - socket文件；p - 管道文件）
- 第二列：文件的链接数目（link number），后面学到文件系统的时候再细说。
- 第三列：文件的用户属主（user owner）
- 第四列：文件的用户组属主（group owner）
- 第五列：文件的大小（size），单位为字节（Bytes）
- 第六列：文件的最后修改时间（last-modified time）
- 第七列：文件名（file name）

#### $\S$ 文件权限

$$
d\;\underbrace{\text{rwx}}_{\text{owner}} \quad \underbrace{\text{r-x}}_{\text{group}} \quad \underbrace{\text{---}}_{\text{others}}
$$
文件类型后面的9位用来表示文件的权限：
- 每3位表示其中一组用户的权限：
    * u - owner用户属主; 
    * g - group owner用户组属主; 
    * o - others 其他用户
- 其中权限的表示：
    * `r`表示读权限（read），显示用户可用`cat`等命令读取文件内容；
    * `w`表示写权限 (write)，显示用户可以编辑和修改某个文件；
    * `x`表示执行权限 (execute)，显示用户可以执行该文件（该文件为二进制或者脚本文件）；
    * 对应位置上为`-`则表示改组用户没有该权限
    
<font color="red">$\S$注意</font>：

对于一个目录，用户必须同时具有读权限和执行权限才可以打开并查看其中的文件；而对该目录必须具有写权限才能在其中添加和删除文件、目录。

#### <font color="red">$\S$特殊文件权限</font>

我们查看一下目录`/tmp`以及可执行文件`/usr/bin/passwd`的权限，发现有什么特殊的没有？
```
drwxrwxrwt. 25 root root    4096 1月   9 13:17 /tmp
-rwsr-xr-x.         1 root root 27832 5月   3  2014 /usr/bin/passwd
```

- 不难发现，对`/tmp`目录，其`others`的权限为`rwt`，这里的`t`表示`sticky-bit`，任意用户可以在该目录下创建文件，修改属于自身的文件，但不能读取和写、执行属于其他用户的文件（`root`除外）；
- 而对于文件`/usr/bin/passwd`，其`owner`的权限为`rws`，这里的`s`表示`super-user`，也就是说当普通用户在执行`passwd`时，临时获得了该文件`owner`，也就是`root`的临时权限，这样就可以修改密码文件`/etc/passwd`和`/etc/shadow`了。

### 2.2 修改文件权限

$$
\underbrace{\text{rwx}}_{\underbrace{111}_{7}}\quad  \underbrace{\text{r-x}}_{\underbrace{101}_{5}} \quad \underbrace{\text{r--}}_{\underbrace{100}_{4}}
$$

- 文件的权限可用3位八进制表示，也就是说每三位rwx可表示为0-7的数字，如`rwx`表示为二进制`111`，也就是对应于八进制`7`；而`r-x`表示为二进制`101`，也就是对应于八进制`5`；因此`rw-r--r--`用八进制表示为`644`；
- 我们可以增加权限，也可以去除权限，还可以分配权限，如
    * `chmod u+x filename`表示为文件属主（`u`）增加（`+`）执行（`x`）的权限；
    * `chmod o-r filename` 表示为其他用户（`o`）去除（`-`）读（`r`）的权限；
    * `chmod g=r-x filename`表示将文件的用户组属主（`g`）赋予（`=`）读和执行（`r-x`）的权限；
    * `chmod 755 filename`表示将文件的权限修改为`rwxr-xr-x`。

### 2.3 修改文件属主

- `chown`可以同时修改文件的用户属主（user owner）和用户组属主（group owner）；
- `chgrp`只能修改文件的用户组属主（group owner）。

### <font color="red">$\S$ 上机练习2</font>

能不能创建目录`/app`，并修改权限，实现用户`bio`和`bio2`能相互编辑、修改各自在该目录下创建的文件？